# Synchronous Generator dq 7th order model vs transient stability classical model

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import dpsimpy

#matplotlib widget

## EMT Simulation

### Parametrization

In [ ]:
# General grid parameters
V_nom_MV = 24e3
V_nom_HV = 230e3
nom_freq = 60
ratio = V_nom_MV / V_nom_HV
nom_omega = nom_freq * 2 * math.pi

# Machine parameters synchronous generator
H = 3.7
Rs = 0.003
Ld = 1.8099
Lq = 1.7600
Ld_t = 0.2999
Lq_t = 0.6500
Ld_s = 0.2299
Lq_s = 0.2500
Ll = 0.15
Td0_t = 8.0669
Tq0_t = 0.9991
Td0_s = 0.0300
Tq0_s = 0.0700

# Operation point synchronous generator 
set_point_active_power = 300e6
set_point_voltage = 1.05*V_nom_MV

# Breaker
breaker_open = 1e9
breaker_closed = 0.001

# Line 
line_lenght = 100
line_resistance = 1.27e-4 * 529.0 * line_lenght * ratio**2
line_inductance = 9.05e-4 * 529.0 * line_lenght * ratio**2 / nom_omega
line_capacitance = (1.81e-3 / 529.0 * line_lenght / ratio**2) / nom_omega
line_conductance = 8e-2

# Simulation parameters
sim_name = "EMT_SynGenDQ7odTrapez_OperationalParams_SMIB_Fault";
final_time = 1.0
time_step = 10e-6
start_time_fault = 0.2

### Powerflow for Initialization

In [ ]:
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)
time_step_pf = final_time
final_time_pf = final_time + time_step_pf

# Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

# Synchronous generator ideal model
gen_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen", dpsimpy.LogLevel.debug)
gen_pf.set_parameters(rated_apparent_power=555e6, rated_voltage=24e3,
                        set_point_active_power=set_point_active_power,
                        set_point_voltage=set_point_voltage,
                        powerflow_bus_type=dpsimpy.PowerflowBusType.PV)
gen_pf.set_base_voltage(V_nom_MV)
gen_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PV)

# Grid bus as Slack
extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(V_nom_MV)
extnet_pf.set_base_voltage(V_nom_MV)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

# Line
line_pf = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_pf.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)
line_pf.set_base_voltage(V_nom_MV)

# Topology
gen_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
extnet_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(nom_freq, [n1_pf, n2_pf], [gen_pf, line_pf, extnet_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v1", "v", n1_pf)
logger_pf.log_attribute("v2", "v", n2_pf)
logger_pf.log_attribute("v_line", "v_intf", line_pf)
logger_pf.log_attribute("i_line", "i_intf", line_pf)
logger_pf.log_attribute("v_gen", "v_intf", gen_pf)
logger_pf.log_attribute("ig", "i_intf", gen_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic Simulation

In [ ]:
dpsimpy.Logger.set_log_dir("logs/"+sim_name)

# Extract relevant powerflow results
init_terminal_volt = np.abs(n1_pf.single_voltage() * dpsimpy.RMS3PH_TO_PEAK1PH)
init_volt_angle = np.angle(n1_pf.single_voltage())
init_active_power = np.real(gen_pf.get_apparent_power())
init_reactive_power = np.imag(gen_pf.get_apparent_power())
init_mech_power = init_active_power

# Nodes
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)

# Components
# Synch
gen = dpsimpy.emt.ph3.SynchronGeneratorDQTrapez("SynGen", dpsimpy.LogLevel.debug)
gen.set_parameters_operational_per_unit(
    nom_power=555e6, nom_volt=24e3, nom_freq=60, pole_number=2, nom_field_cur=1300,
    Rs=Rs, Ld=Ld, Lq=Lq, Ld_t=Ld_t, Lq_t=Lq_t, Ld_s=Ld_s, Lq_s=Lq_s, Ll=Ll, Td0_t=Td0_t, Tq0_t=Tq0_t,
    Td0_s=Td0_s, Tq0_s=Tq0_s, inertia=H, init_active_power=init_active_power, init_reactive_power=init_reactive_power,
    init_terminal_volt=init_terminal_volt, init_volt_angle=init_volt_angle, init_field_voltage=7.0821,
    init_mech_power=init_mech_power
    
)

# Grid bus as Slack
extnet = dpsimpy.emt.ph3.NetworkInjection("Slack", dpsimpy.LogLevel.debug)

# Line
line = dpsimpy.emt.ph3.PiLine("PiLine", dpsimpy.LogLevel.debug)
line.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(line_resistance),
                       dpsimpy.Math.single_phase_parameter_to_three_phase(line_inductance),
                       dpsimpy.Math.single_phase_parameter_to_three_phase(line_capacitance),
                       dpsimpy.Math.single_phase_parameter_to_three_phase(line_conductance))

# Breaker
fault = dpsimpy.emt.ph3.Switch("Br_fault", dpsimpy.LogLevel.debug)
fault.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(breaker_open),
                     dpsimpy.Math.single_phase_parameter_to_three_phase(breaker_closed))
fault.open()

# Topology
gen.connect([n1])
line.connect([n1, n2])
extnet.connect([n2])
fault.connect([dpsimpy.emt.SimNode.gnd, n1])
system = dpsimpy.SystemTopology(nom_freq, [n1, n2], [gen, line, fault, extnet])


# Initialization of dynamic topology
system.init_with_powerflow(systemPF=system_pf, domain=dpsimpy.Domain.EMT)

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("v_line", "v_intf", line)
logger.log_attribute("i_line", "i_intf", line)
logger.log_attribute("v_gen", "v_intf", gen)
logger.log_attribute("i_gen", "i_intf", gen)
logger.log_attribute("wr_gen", "w_r", gen)
logger.log_attribute("delta_r", "delta_r", gen)

# Events
sw1 = dpsimpy.event.SwitchEvent3Ph(start_time_fault, fault, True)

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.add_event(sw1)
sim.run()

## DP Simulation

### Parametrization

In [ ]:
# Power System
V_nom = 230e3

# Generator
nom_power = 500e6
nom_ph_ph_volt_RMS = 22e3
nom_freq = 60
nom_omega = nom_freq * 2 * math.pi
H = 5
Xpd = 0.31
Rs = 0.003 * 0
D=1.5

# Initialization parameters
init_mech_power = 300e6
init_active_power = 300e6
set_point_voltage = nom_ph_ph_volt_RMS * 1.05

# Transformer
t_ratio = V_nom / nom_ph_ph_volt_RMS

# PiLine parameters calculated from CIGRE Benchmark system
line_resistance = 6.7
line_inductance = 47.0/nom_omega
line_capacitance = 3.42e-4/nom_omega
line_conductance = 0

# Parameters for powerflow initialization
# Slack voltage: 1pu
V_slack = V_nom

# Switch to trigger fault at generator terminal
switch_open = 1e6
switch_closed = 0.1

# Simulation parameters
sim_name = "DP_SynGenTrStab_SMIB_Fault";
final_time = 20
time_step = 0.001
start_fault_event = True
end_fault_event = True
start_time_fault = 10
end_time_fault = 10.2
cmd_Inertia = 1.0
cmd_Damping = 1.0

### Powerflow for Initialization

In [ ]:
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)
time_step_pf = final_time
final_time_pf = final_time + time_step_pf

# Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

# Synchronous generator ideal model
gen_pf = dpsimpy.sp.ph1.SynchronGenerator("Generator", dpsimpy.LogLevel.debug)
gen_pf.set_parameters(rated_apparent_power=nom_power, rated_voltage=nom_ph_ph_volt_RMS,
                        set_point_active_power=init_active_power,
                        set_point_voltage=set_point_voltage*t_ratio,
                        powerflow_bus_type=dpsimpy.PowerflowBusType.PV)
gen_pf.set_base_voltage(V_nom)
gen_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PV)

# Grid bus as Slack
extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(V_slack)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

# Line
line_pf = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_pf.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)
line_pf.set_base_voltage(V_nom)

# Switch
fault_pf = dpsimpy.sp.ph1.Switch("Br_fault", dpsimpy.LogLevel.debug)
fault_pf.set_parameters(switch_open, switch_closed)
fault_pf.open()

# Topology
gen_pf.connect([n1_pf])
fault_pf.connect([dpsimpy.sp.SimNode.gnd, n1_pf])
line_pf.connect([n1_pf, n2_pf])
extnet_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(nom_freq, [n1_pf, n2_pf], [gen_pf, line_pf, extnet_pf, fault_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v1", "v", n1_pf)
logger_pf.log_attribute("v2", "v", n2_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic simulation

In [ ]:
sim_name_dp = sim_name + "_DP" 
dpsimpy.Logger.set_log_dir("logs/"+sim_name_dp)

# Nodes
n1_dp = dpsimpy.dp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_dp = dpsimpy.dp.SimNode("n2", dpsimpy.PhaseType.Single)

# Components
gen_dp = dpsimpy.dp.ph1.SynchronGeneratorTrStab("SynGen", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen_dp.set_standard_parameters_PU(
    nom_power=nom_power, nom_volt = nom_ph_ph_volt_RMS, nom_freq=nom_freq,
    Xpd = Xpd*t_ratio**2, inertia=cmd_Inertia*H, Rs=Rs, D=cmd_Damping*D)

init_apparent_power = gen_pf.get_apparent_power()
gen_dp.set_initial_values(init_apparent_power, init_mech_power)

# Grid bus as Slack
extnet_dp = dpsimpy.dp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)

# Line
line_dp = dpsimpy.dp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_dp.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)

# Switch
fault_dp = dpsimpy.dp.ph1.varResSwitch("Br_fault", dpsimpy.LogLevel.debug)
fault_dp.set_parameters(switch_open, switch_closed)
fault_dp.set_init_parameters(time_step)
fault_dp.open()

# Topology
gen_dp.connect([n1_dp])
line_dp.connect([n1_dp, n2_dp])
extnet_dp.connect([n2_dp])
fault_dp.connect([dpsimpy.dp.SimNode.gnd, n1_dp])
system_dp = dpsimpy.SystemTopology(nom_freq, [n1_dp, n2_dp], [gen_dp, line_dp, fault_dp, extnet_dp])


# Initialization of dynamic topology
system_dp.init_with_powerflow(system_pf)

# Logging
logger_dp = dpsimpy.Logger(sim_name_dp)
logger_dp.log_attribute("v1", "v", n1_dp)
logger_dp.log_attribute("v2", "v", n2_dp)

logger_dp.log_attribute("v_line", "v_intf", line_dp)
logger_dp.log_attribute("i_line", "i_intf", line_dp)

logger_dp.log_attribute("v_slack", "v_intf", extnet_dp)
logger_dp.log_attribute("i_slack", "i_intf", extnet_dp)

logger_dp.log_attribute("i_fault", "i_intf", fault_dp)

logger_dp.log_attribute("Ep", "Ep", gen_dp)
logger_dp.log_attribute("v_gen", "v_intf", gen_dp)
logger_dp.log_attribute("i_gen", "i_intf", gen_dp)
logger_dp.log_attribute("wr_gen", "w_r", gen_dp)
# Name changed from CPP example to work in comparison below
logger_dp.log_attribute("delta_r", "delta_r", gen_dp)
logger_dp.log_attribute("P_elec", "P_elec", gen_dp)
logger_dp.log_attribute("P_mech", "P_mech", gen_dp)

# Simulation
sim_dp = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step)
sim_dp.set_final_time(final_time)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.add_logger(logger_dp)
sim_dp.do_system_matrix_recomputation(True)


# Events
if start_fault_event:
    sw1 = dpsimpy.event.SwitchEvent(start_time_fault, fault_dp, True)
    sim_dp.add_event(sw1)
    
if end_fault_event:
    sw2 = dpsimpy.event.SwitchEvent(end_time_fault, fault_dp, False)
    sim_dp.add_event(sw2)
    
sim_dp.run()

## Results 3ph EMT 7th order model with trapezoidal rule

In [ ]:
work_dir = 'logs/EMT_SynGenDQ7odTrapez_OperationalParams_SMIB_Fault/'
log_name = 'EMT_SynGenDQ7odTrapez_OperationalParams_SMIB_Fault'
print(work_dir + log_name + '.csv')
ts_emt3ph_DQ7odTrapez = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Results 1ph DP transient stability classical model

In [ ]:
work_dir = 'logs/DP_SynGenTrStab_SMIB_Fault_DP/'
log_name = 'DP_SynGenTrStab_SMIB_Fault_DP'
print(work_dir + log_name + '.csv')
ts_dp1ph_TrStab = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Generator EMF

In [ ]:
plt.style.use('default')
#plt.rcParams.update({'font.size': 22})

timestep=10e-6;
t_begin=0
t_end=5

# Fault at t=0.1 s untill t=0.15s
begin_idx = int(t_begin/timestep)
end_idx= int(t_end/timestep)

plt.figure(figsize=(12,8))
plt.ylabel('Generator emf (V)', fontsize=18)

for name in ['Ep']:
    #plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).values[begin_idx:end_idx], label=name + '  Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + '  Classical model (2nd order)', linestyle='--')
    
plt.legend()
plt.show()

## Generator terminal voltage

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal voltage (V)', fontsize=18)

for name in ['v_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).values[begin_idx:end_idx], label=name + '  Full model (9th order)')
    #plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + '  Classical model (2nd order)', linestyle='--')
    
plt.legend(fontsize=14)
plt.show()

## Genrerator terminal Current

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal current (V)', fontsize=18)

for name in ['i_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).values[begin_idx:end_idx], label=name + '  Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + '  Classical model (2nd order)', linestyle='--')

plt.legend()
plt.show()

## Voltage across line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['v_line']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).values[begin_idx:end_idx], label=name + '  Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + '  Classical model (2nd order)', linestyle='--')

plt.legend()
plt.show()

## Current through line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['i_line']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].interpolate(timestep).values[begin_idx:end_idx], label=name + '  Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + '  Classical model (2nd order)', linestyle='--')

plt.legend()
plt.show()

## Rotor frequency

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Rotor frequency (Hz)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.ylim(59,61)

#if x_axis limits are changed above, change them again to consider the complete duration

#ROCOF
for name in ['wr_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name].interpolate(timestep).time[begin_idx:end_idx], ts_emt3ph_DQ7odTrapez[name].interpolate(timestep).values[begin_idx:end_idx]*60, label='Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='Classical model (2nd order)', linestyle='--')

plt.legend(fontsize=18)
plt.show()

## Rotor angular velocity $\omega _r$

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Rotor angular velocity (rad/s)', fontsize=18)


for name in ['wr_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name].interpolate(timestep).time[begin_idx:end_idx], 2*math.pi*60*ts_emt3ph_DQ7odTrapez[name].interpolate(timestep).values[begin_idx:end_idx], label=name +' Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' Classical model (2nd order)' , linestyle='--')

plt.legend()
plt.show()

## Rotor angle $\delta _r$

In [ ]:
plt.figure(figsize=(12,8))
for name in ['delta_r']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name].interpolate(timestep).time[begin_idx:end_idx], ts_emt3ph_DQ7odTrapez[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label=name + ' Full model (9th order)')
    plt.plot(ts_dp1ph_TrStab[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label=name + ' Classical model (2nd order)', linestyle='--')
